In [5]:
import warnings
warnings.filterwarnings(action='ignore')

In [6]:
import torch
print("torch:", torch.__version__, 
      "cuda:", torch.version.cuda, 
      "is_available:", 
      torch.cuda.is_available())

torch: 2.6.0+cu124 cuda: 12.4 is_available: True


In [10]:
import os 
import requests
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

True

## Inference API

In [ ]:
import os
from huggingface_hub import InferenceClient

client = InferenceClient(
    provider="hf-inference",
    api_key=os.environ["HF_TOKEN"],
)

result = client.translation(
    "Меня зовут Вольфганг и я живу в Берлине",
    model="Helsinki-NLP/opus-mt-ko-en",
)

## pipeline()
task: 텍스트 기반 감성분석, 분류(제로샷), 생성

In [14]:
from transformers import pipeline
classifier = pipeline("text-classification") #task지정
classifier("I want to learn using the refrigerator")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cuda:0


[{'label': 'NEGATIVE', 'score': 0.9935249090194702}]

## fine tuning

In [16]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis", "tabularisai/multilingual-sentiment-analysis") #task지정
classifier("냉장고를 이용하여 공부하는 것은 즐겁다")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cuda:0


[{'label': 'Negative', 'score': 0.3945055902004242}]

In [ ]:
from transformers import pipeline
classifier = pipeline("fill-mask", "bert-base-multilingual-cased") #task지정
classifier("People often believe that happiness comes from getting what they want. However, studies show that true happiness is not about having everything, but about being able to [ ]  what you already have.")

Device set to use cuda:0


[{'label': 'Negative', 'score': 0.5140126347541809}]

## question-answering

In [ ]:
# 개체명 인식

In [ ]:
# summarization 문장요약

In [ ]:
# 한글->영어 번역

In [ ]:
# image to text